# Deep Bucket Lab: 
## A playground for understanding hydrologic process representations with Deep Learning

This notebook is designed to help students understand the fundamentals of simulating dynamic systems with Deep Learning. 


The tutorial models a leaking bucket, a much simpler hydrological system than a watershed, allowing us to easily evaluate whether our leaking bucket models approximante expected physical processes.

We generate all data synthetically in this notebook using simple numerical simulations of the hydrological system (leaking buckets) and varying bucket characteristics/attributes and variables including dynamic forcings for precipitation. 

We develop a single-layer Long Short-Term Memory (LSTM) network to learn dynamics of water level and fluxes out from bucket based on the "ground truth" data.

In this notebook, the user can experiment with modifying the bucket model attributes and modelling setup to explore hydrologic process representations and their predictability.

# 1. Setup

The first thing to do is setup the notebook environment with all the libraries, declare model global parameters, settings, variables and functions that define the bucket system we want to represent, as well as define the hyperparameters and structure for the deep learning model. 

Note: in a typical full-scale modeling framework, these would be decalared in a configuration file.

### 1.1 Importing libraries 

We import standard libraries for data management, calculations and plotting. 

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

We import machine learning libraries for modeling architectures, training, validation and testing.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torch.autograd import Variable 
import sklearn
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import trange, tqdm

### 1.2 Defining the physical bucket model system

**Global variables**

Constants in our bucket model system are:
- g, gravitational acceleration [$m/s^2$]
- time step [$s$]

In [6]:
g = 9.807
time_step = 1

**Parameters for the Forcing Process (Precipitation)**

To generate precipitation data for our hydrological system, we employ a simple random process that determines whether it is raining based on the previous state, as well as the total amount of rain during the event.

We classify precipitation into three types: None, Light, and Heavy. Each type has distinct ranges of rainfall probabilities and depths.

In [7]:
rain_probability_range = {"None": [0.6, 0.7], 
                          "Light": [0.5, 0.8], 
                          "Heavy": [0.2, 0.3]}

rain_depth_range = {"Light": [0, 2], "Heavy": [2, 8]}

**Bucket physical attributes**

The physical leaky bucket attributes include:
- A_bucket, bucket area [$m^2$]
- A_spigot, spigot areas [$m^2$]
- H_bucket, bucket heights [$m$]
- H_spigot, spigot heights [$m$]
- K_infiltration, infiltration rate [$mm/hr$]
- ET_parameter, evapotranspiration parameter [$mm/day$]

We will generate a diversity of leaky buckets by randomy selecting their physical attribute values from the following possible ranges.

In [8]:
bucket_attributes_range = {"A_bucket": [1.0, 2.0],
                           "A_spigot": [0.1, 0.2],
                           "H_bucket": [5.0, 6.0],
                           "H_spigot": [1.0, 3.0],
                           "K_infiltration": [1e-7, 1e-9],
                           "ET_parameter": [7, 9]
                          }

bucket_attributes_list = list(bucket_attributes_range.keys())

**Channel physical parameters**

This section includes the channel physical parameters. Initially, the transformation parameters are defined

- transform_type: 0 (null transform), 1 (simple shift), 2 (time-based displacement), 3 (simple attenuation), 4 (LagK), 5 (Muskingum-Cunge)
- shift_amount: position number to shift (applies when transform_type == 1)
- time_shift: specified number of seconds to move the streamflow
- attenuation_factor: value for multiplying the streamflow
- lag_amount: number of streamflow positions to delay the output
- param_list: list of parameters to apply Muskingum-Cunge transformation


In [9]:
transform_params_range = {"transform_type": [0, 1, 3, 4], #transform_types 2 and 5 are not currently implemented
                          "shift_amount": [0, 100],
                          # "time_shift": [],
                          "attenuation_factor": [0.01, 0.99],
                          "lag_amount": [0, 100]
                         }

transform_params_list = list(transform_params_range.keys())

**Model input and output variables**

The data fluxes in and out of the leaky bucket considered in this stystem and the leaky bucket state, include
- precipitation into the bucket as a model input (precip) 
- the actual and potential loss to evaporation from the bucket as model inputs (et, pet)
- the water flow over the bucket and out of the spigot as a simulated model output(q_overflow, q_spigot)
- and the state of the water head in the bucket as a simluated model output (h_bucket).

We define lists based on these input and output leaking bucket model variables.

In [10]:
input_vars = ['precip', 'et']
input_vars.extend(bucket_attributes_list)
output_vars = ['q_overflow', 'q_spigot']
n_input = len(input_vars)
n_output = len(output_vars)

**Data noise**

Because real world systems are noisy, we can add noise to the synthetic data by multiplying the values by a random factor taken from a normal distribution with a mean of 1 and a standard deviation prescribed for different variables.

In [11]:
is_noise = True
noise = {"pet": 0.1, "et": 0.1, "q": 0.1, "head": 0.1} 

Illustration of leaking bucket hydrological system


![title](figs/bucket_schematic.png)

### 1.3 Defining the modeling setup 

**Deep Learning Model (LSTM) Hyperparameters**

The hyperparameters for the LSTM (Long Short-Term Memory) deep learning model include:
- device: The device (CPU or CUDA) used for training and inference.
- hidden_state_size: The number of units in the hidden state of the LSTM model.
- num_layers: Number of LSTM layers.
- num_epochs: Number of training epochs.
- batch_size: Number of samples in each training batch.
- seq_length: Length of input sequences.
- learning_rate: Learning rate for the optimizer.
- num_classes: Number of output classes.
- input_size: Size of the input layer.

These hyperparameters control the behavior and performance of the LSTM model and can be adjusted to optimize the model's accuracy and generalization capabilities for specific tasks and datasets.

In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using CUDA device: ", torch.cuda.get_device_name(0))
else:
    device = 'cpu'
hidden_state_size = 16
num_layers = 8
num_epochs = 5  
batch_size = 256 
seq_length = 24
learning_rate = np.linspace(start=0.1, stop=0.01, num=num_epochs)

**Size of data records and splits**

The deep learning framework requires the data to be split into 3 different sets
- Back propogation is done on the training set (train)
- Hyperparameter tuning is done on the validation set (val)
- Calculating model accuracy is done on the testing set (test)

We will define how many randomly generated bucket configurations to use for the "train", "val", and "test" sets, as well as the length of the simulations for each set.

For example, if we have a training set with 10 different buckets, the "split" refers to dividing the data into multiple bucket configurations. In this case, it means that we will train the model using 10 randomly generated bucket configurations.

Furthermore, if we have 1000 time steps for each bucket, the "time split" indicates the number of time steps we will use for training the model. In this scenario, we will train the model using 1000 time steps for each of the 10 bucket configurations.

**For creating a basin network**

We will define three basin network sets for the use of "train", "val", and "test". Additionally it is necessary to define the number of buckets in every basin network (n_bucket_per_network). Consequently, the number of buckets is equal to the number of basin networks multiplied by the number of buckets per network.

In [13]:
#n_buckets_split = {"train": 10, "val": 5,"test": 1}
time_splits = {"train": 1000, "val": 500,"test": 500}

n_buckets_per_network = 3
n_basin_networks_split = {"train": 15, "val": 7, "test": 2}
n_buckets_split = {"train": n_basin_networks_split["train"] * n_buckets_per_network, "val": n_basin_networks_split["val"] * n_buckets_per_network,"test": n_basin_networks_split["test"] * n_buckets_per_network}

We use the settings above to calculate the total length of the data record to generate and the total number of buckets and basin networks

In [14]:
num_records = time_splits["train"] + time_splits["val"] + time_splits["test"] + seq_length * 3
n_buckets = n_buckets_split["train"] + n_buckets_split["val"] + n_buckets_split["test"]
n_basin_networks = n_basin_networks_split["train"] + n_basin_networks_split["val"] + n_basin_networks_split["test"]

We then calculate the parameters for the bucket and time splits necessary to feed the model with separate datasets for training, validation, and testing.

We need to create a new function to calculate parameter for the basin network splits into separate datasets.

In [15]:
def split_parameters():
    # create lists of bucket indices for each set based on the given bucket splits
    buckets_for_training = list(range(0, n_buckets_split['train'] + 1))
    buckets_for_val = list(range(n_buckets_split['train'] + 1, 
                                 n_buckets_split['train'] + n_buckets_split['val'] + 1))
    buckets_for_test = list(range(n_buckets - n_buckets_split['test'], n_buckets))

    # determine the time range for each set based on the given time splits
    train_start = seq_length
    train_end   = time_splits["train"]
    val_start   = train_end + seq_length
    val_end     = val_start + time_splits["val"]
    test_start  = val_end + seq_length
    test_end    = test_start + time_splits["test"]
    
    # organize the split parameters into separate lists for each set
    train_split_parameters = [buckets_for_training, train_start, train_end]
    val_split_parameters = [buckets_for_val, val_start, val_end]
    test_split_parameters = [buckets_for_test, test_start, test_end]
    
    return [train_split_parameters, val_split_parameters, test_split_parameters]

In [16]:
[[buckets_for_training, train_start, train_end],
[buckets_for_val, val_start, val_end],
[buckets_for_test, test_start, test_end]]= split_parameters()
print([[buckets_for_training, train_start, train_end],
[buckets_for_val, val_start, val_end],
[buckets_for_test, test_start, test_end]])

[[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], 24, 1000], [[46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66], 1024, 1524], [[66, 67, 68, 69, 70, 71], 1548, 2048]]


This function will create separate datasets for training, validation, and testing.

In [17]:
def basin_split_param():
    #create lists of basin indices for each set
    basins_train = list(range(0, n_basin_networks_split['train']))
    basins_val = list(range(n_basin_networks_split['train'], 
                            n_basin_networks_split['train'] + n_basin_networks_split['val']))
    basins_test = list(range(n_basin_networks - n_basin_networks_split['test'], n_basin_networks)) 
    
    #place basin split parameters into separate lists
    train_basin_split_param = [basins_train]
    val_basin_split_param = [basins_val]
    test_basin_split_param = [basins_test]
    
    #determine transform parameters per basin network
    
    
    return [train_basin_split_param, val_basin_split_param, test_basin_split_param]

This will run the function above and print out the range for the whole basin network split

In [18]:
[[basin_networks_train],[basin_networks_val],[basin_networks_test]]= basin_split_param()
print([[basin_networks_train],[basin_networks_val],[basin_networks_test]])

[[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]], [[15, 16, 17, 18, 19, 20, 21]], [[22, 23]]]


### 1.4 Creating a sample of diverse buckets

Now that we know how many buckets we want, we can generate a sample of diverse buckets with their respective boundary and initial conditions by randomly sampling from the possible ranges for each attribute defined in the settings above.

In [19]:
def setup_buckets(n_buckets):
    # Boundary conditions
    buckets = {bucket_attribute:[] for bucket_attribute in bucket_attributes_list}
    for i in range(n_buckets):
        for attribute in bucket_attributes_list:
            buckets[attribute].append(np.random.uniform(bucket_attributes_range[attribute][0], 
                                                        bucket_attributes_range[attribute][1]))

    # Initial conditions
    h_water_level = [np.random.random() for i in range(n_buckets)]
    mass_overflow = [np.random.random() for i in range(n_buckets)]
    return buckets, h_water_level, mass_overflow

In [20]:
buckets, h_water_level, mass_overflow = setup_buckets(n_buckets)

### 1.5 Creating the synthetic "precipitation" 

We first randomly assign rainfall parameters for each bucket, using the probability ranges specified for each precipitation type. We then generate synthetic input time series for each bucket model, known as forcing data. This process involves using the previously defined rainfall parameters and the random process defined in the function below.

In [21]:
def pick_rain_params():
    buck_rain_params = [rain_depth_range,
                        np.random.uniform(rain_probability_range["None"][0],
                                            rain_probability_range["None"][1]),
                        np.random.uniform(rain_probability_range["Heavy"][0],
                                            rain_probability_range["Heavy"][1]),
                        np.random.uniform(rain_probability_range["Light"][0],
                                            rain_probability_range["Light"][1])
                 ]
    return buck_rain_params

In [22]:
def random_rain(preceding_rain, bucket_rain_params):
    depth_range, no_rain_probability, light_rain_probability, heavy_rain_probability = bucket_rain_params
    # some percent of time we have no rain at all
    if np.random.uniform(0.01, 0.99) < no_rain_probability:
        rain = 0

    # When we do have rain, the probability of heavy or light rain depends on the previous day's rainfall
    else:
        # If yesterday was a light rainy day, or no rain, then we are likely to have light rain today
        if preceding_rain < depth_range["Light"][1]:
            if np.random.uniform(0, 1) < light_rain_probability:
                rain = np.random.uniform(0, 1)
            else:
                # But if we do have heavy rain, then it could be very heavy
                rain = np.random.uniform(depth_range["Heavy"][0], depth_range["Heavy"][1])

        # If it was heavy rain yesterday, then we might have heavy rain again today
        else:
            if np.random.uniform(0, 1) < heavy_rain_probability:
                rain = np.random.uniform(0, 1)
            else:
                rain = np.random.uniform(depth_range["Light"][0], depth_range["Light"][1])
    return rain

We generate a random rainfall input timeseries for each bucket and store it in a dictionary

In [23]:
in_list = {}
for ibuc in range(n_buckets):
    bucket_rain_params = pick_rain_params()
    in_list[ibuc] = [0]
    for i in range(1, num_records):
        in_list[ibuc].append(random_rain(in_list[ibuc][i-1], bucket_rain_params))

### 1.6 Running Numerical Simulations of the Bucket Model to Generate "Ground Truth" Data

We run bucket model simulations to generate the data for our system. 

These data represent the "ground truth", which is what we will try to learn with the LSTM. 

We perform numerical simulations of a bucket model for a specific bucket index. It iterates over each time step, updating the water level based on precipitation, evapotranspiration, infiltration, overflow, and spigot outflow. The simulation results, including the water level, overflow, spigot flow, and other attributes, are stored in a data frame. This is a concise representation of the bucket model's behavior over time for the given bucket index.

In [24]:
def run_bucket_simulation(ibuc):
    columns = ['precip', 'et', 'h_bucket', 'q_overflow', 'q_spigot']
    columns.extend(bucket_attributes_list)
    # Memory to store model results
    df = pd.DataFrame(index=list(range(len(in_list[ibuc]))), columns=columns)
    #print(list(range(len(in_list[ibuc]))))
    # Main loop through time
    for t, precip_in in enumerate(in_list[ibuc]):
        
        # Add the input mass to the bucket
        h_water_level[ibuc] = h_water_level[ibuc] + precip_in

        # Lose mass out of the bucket. Some periodic type loss, evaporation, and some infiltration...
        et = np.max([0, (buckets["A_bucket"][ibuc] / buckets["ET_parameter"][ibuc]) * np.sin(t) * np.random.normal(1, noise['pet'])])
        infiltration = h_water_level[ibuc] * buckets["K_infiltration"][ibuc]
        h_water_level[ibuc] = np.max([0 , (h_water_level[ibuc] - et)])
        h_water_level[ibuc] = np.max([0 , (h_water_level[ibuc] - infiltration)])
        if is_noise:
            h_water_level[ibuc] = h_water_level[ibuc] * np.random.normal(1, noise['et'])

        # Overflow if the bucket is too full
        if h_water_level[ibuc] > buckets["H_bucket"][ibuc]:
            mass_overflow[ibuc] = h_water_level[ibuc] - buckets["H_bucket"][ibuc]
            h_water_level[ibuc] = buckets["H_bucket"][ibuc] 
            if is_noise:
                h_water_level[ibuc] = h_water_level[ibuc] - np.random.uniform(0, noise['q'])

        # Calculate head on the spigot
        h_head_over_spigot = (h_water_level[ibuc] - buckets["H_spigot"][ibuc] ) 
        if is_noise:
            h_head_over_spigot = h_head_over_spigot * np.random.normal(1, noise['head'])

        # Calculate water leaving bucket through spigot
        if h_head_over_spigot > 0:
            velocity_out = np.sqrt(2 * g * h_head_over_spigot)
            spigot_out = velocity_out *  buckets["A_spigot"][ibuc] * time_step
            h_water_level[ibuc] = h_water_level[ibuc] - spigot_out
        else:
            spigot_out = 0

        # Save the data in time series
        df.loc[t,'precip'] = precip_in
        df.loc[t,'et'] = et
        df.loc[t,'h_bucket'] = h_water_level[ibuc]
        df.loc[t,'q_overflow'] = mass_overflow[ibuc]
        df.loc[t,'q_spigot'] = spigot_out
        for attribute in bucket_attributes_list:
            df.loc[t, attribute] = buckets[attribute][ibuc]

        mass_overflow[ibuc] = 0
        
    return df

Then, we run and store the simulations for each bucket in a dictionnary

We ran basin network model simulation to create a dictionary that contains n_basin_networks elements. Each element represents a network of n_buckets_per_network buckets.

In [25]:
def run_basin_network_simultation():
    result = {}
    result['buckets'] = {}
    for i in range(n_basin_networks):
        bucket_dictionary = {}
        for j in range(n_buckets_per_network):
            bucket_dictionary[j] = run_bucket_simulation((i * n_buckets_per_network) + j)
        result['buckets'][i] = bucket_dictionary
    return result

In [ ]:
%%time
basin_network_dictionary = {}
basin_network_dictionary = run_basin_network_simultation()
#print(basin_network_dictionary)

In [ ]:
def sum_q_bucket(bucket_df):
    return bucket_df['q_overflow']+bucket_df['q_spigot']

def sum_q_network(network_dict):
    for k, bucket in network_dict.items():
        bucket['q_total']=sum_q_bucket(bucket)
        

In [ ]:
for k, network in basin_network_dictionary['buckets'].items():
    sum_q_network(network)

In [ ]:
#print(basin_network_dictionary)
#df = pd.DataFrame.from_dict(basin_network_dictionary)
#df.to_csv('basin_network_data.csv', index=False, header=True)

#df1 = pd.DataFrame.from_dict(basin_network_dictionary['buckets'][0][2])
#df1.to_csv('buckets.csv', index=False, header=True)

In [ ]:
# Get the simulation results for the first basin network (index 0)
basin_network_index = 3
bucket_results = basin_network_dictionary['buckets'][basin_network_index]
# Plot the simulation results for each bucket in the basin network
for bucket_index, df in bucket_results.items():
    time_steps = range(len(df))
    # Plot the water level in the bucket
    plt.plot(time_steps, df['h_bucket'], label=f'Bucket {bucket_index}' )
# Add labels and legend
plt.xlabel('Time Step')
plt.ylabel('Water Level')
plt.title(f'Simulation Results for Basin Network {basin_network_index}')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# %%time
# result = run_basin_network_simultation()

In [ ]:
# %%time
# plot_diagram(result)

In [ ]:
# result['buckets'][3][0]['q_total']= result['buckets'][3][0]['q_overflow']+result['buckets'][3][0]['q_spigot']

In [1]:
 #result['buckets'][3][0]

NameError: name 'result' is not defined

### 1.7 Create a combined streamflow

We create a function to transform streamflow. The transformation type to perform can be one of the following:

- **Null transformation:** Returns the same series of input streamflows
- **Simple shift:** Move the elements right or left according to parameters shift_amount
- **Time-based shift:** Move the elements to the right or left according to the specified number of seconds time_shift; requires some information about the internal time values of the series.
- **Simple Attenuation:** multiply each element of the series by an attenuation factor
- **LagK:** Move each element to position K (lag_amount)
- **Muskingum-Cunge:** Muskingum-Cunge method to combine river flows

In [32]:
#def mcTransform(Q, param_list):
#    return Q

def transformQ(Q, T):
    # Verify if the transformation parameters are null
    if T[0] == 0 or T[0] is None:
        # The null transformation returns the same series of input streamflows
        return Q
    else:
        transform_type = T[0]
        if transform_type == 1:
            # Implement the simple shift transformation
            # Move the elements to the right or left according to parameters
            shift_amount = T[1]
            #print(shift_amount)
            #print(Q[shift_amount:])
            #print(Q[:shift_amount])
            #transformed_Q = Q[shift_amount:] + Q[:shift_amount]
            transformed_Q = np.roll(Q, shift_amount).tolist()
        elif transform_type == 2:
            # Implementing time-based displacement transformation
            # Move the elements to the right or left according to the specified number of seconds
            time_shift = T[1]
            # Information about the internal time values of the series should be used here
            # to calculate the appropriate displacement based on the specified seconds
            # transformed_Q = ...
            # Implement the code for this transformation according to needs and available data
        elif transform_type == 3:
            # Implement the simple attenuation transformation
            # Multiplying each element of the series by an attenuation factor
            attenuation_factor = T[1]
            transformed_Q = [q * attenuation_factor for q in Q]
        elif transform_type == 4:
            # Implement LagK transformation
            # Move each element to position K
            lag_amount = T[1]
            transformed_Q = [Q[i - lag_amount] if i >= lag_amount else 0 for i in range(len(Q))]
        elif transform_type == 5:
            # Implement the Muskingum-Cunge method to combine river flows
            param_list = T[1]
            # transformed_Q = mcTransform(Q, param_list)
            # Implement the code for this transformation according to needs and available data
        #else:
            # In case new transformations are added in the future, they can be handled here
            # by adding more elif blocks
        return transformed_Q


Now we combine the transformed streamflows of a basin network in one single streamflow

In [33]:
def get_param_key(transform_type):
    if transform_type == 0:
        return None
    elif transform_type == 1:
        return "shift_amount"
    #elif transform_type == 2:  # For time-based transformation (not implemented yet)
    #    return "time_shift"
    elif transform_type == 3:
        return "attenuation_factor"
    elif transform_type == 4:
        return "lag_amount"
    #elif tranform_type == 5:  # For Muskingum-Cunge transformation (not implemented yet)
    #    return "param_list"

In [34]:
def generate_transform_params(bucket_basin):
    n = len(bucket_basin)
    transform_params = []
    for i in range(n - 1):
        transform_type = np.random.choice(transform_params_range["transform_type"])
        param_key = get_param_key(transform_type)
        if transform_type == 0:
            param_value = None
        else:
            param_range = transform_params_range[param_key]
            #param_value = None
            if param_key in ["shift_amount", "lag_amount"]:
                param_value = np.random.randint(param_range[0], param_range[1] + 1)
            else:
                param_value = np.random.uniform(param_range[0], param_range[1])
        transform_params.append((transform_type, param_value))
    transform_params.append((0, None))
    return transform_params

In [35]:
def generate_transform_params_network():
    transform_params_vector = []
    for i in range(n_basin_networks):
        transform_params_vector.append([i, generate_transform_params(basin_network_dictionary['buckets'][i])])
    return transform_params_vector

In [36]:
transform_params_vector = generate_transform_params_network()

In [37]:
print(transform_params_vector)

[[0, [(4, 29), (3, 0.4574626099854994), (0, None)]], [1, [(4, 27), (3, 0.44019956014266015), (0, None)]], [2, [(1, 28), (1, 73), (0, None)]], [3, [(1, 14), (1, 69), (0, None)]], [4, [(4, 27), (4, 96), (0, None)]], [5, [(0, None), (3, 0.10604508323484506), (0, None)]], [6, [(1, 22), (3, 0.3418292640751636), (0, None)]], [7, [(1, 60), (3, 0.7146061388710315), (0, None)]], [8, [(0, None), (1, 20), (0, None)]], [9, [(3, 0.3544260221983905), (3, 0.5638987641891546), (0, None)]], [10, [(4, 13), (4, 65), (0, None)]], [11, [(4, 48), (3, 0.6776561821506532), (0, None)]], [12, [(0, None), (4, 75), (0, None)]], [13, [(1, 62), (3, 0.5587372555372387), (0, None)]], [14, [(3, 0.12564078298948353), (3, 0.6595240974511546), (0, None)]], [15, [(4, 1), (1, 42), (0, None)]], [16, [(0, None), (1, 83), (0, None)]], [17, [(4, 58), (3, 0.5273495111924171), (0, None)]], [18, [(0, None), (1, 11), (0, None)]], [19, [(3, 0.7673139886963963), (3, 0.14906103307046048), (0, None)]], [20, [(0, None), (0, None), (0, 

In [38]:
def combine_streamflows(flows, transform_params):
    transformed_flows = []
    for Q, T in zip(flows, transform_params):
        transformed_flows.append(transformQ(Q, T))
    '''
    print("Flows:")
    print(flows)
    print("Transform_params:")
    print(transform_params)
    print("Transformed_Flows:")
    print(transformed_flows)
    print("suma:")
    print(np.sum(transformed_flows, axis=0))
    '''
    return np.sum(transformed_flows, axis=0)

In [51]:
def streamflows_output(network_collection):
    combined_streamflows_collection = {}
    for i, (k, network) in enumerate(network_collection.items()):  # We should iterate on objects consistently -- this is messy.
        q_total_vector = []
        for bucket in network.values():
            # print(bucket['q_total'])
            q_total_vector.append(bucket['q_total'])
        combined_streamflows_collection[k] = {}
        combined_streamflows_collection[k]['q_total_output'] = combine_streamflows(q_total_vector, transform_params_vector[i][1])
    return combined_streamflows_collection

In [55]:
combined_streamflows_dictionary = streamflows_output(basin_network_dictionary['buckets'])

In [64]:
len(combined_streamflows_dictionary)

24

In [ ]:
'''
Q1 = [0.4, 0.5, 0.6]
Q2 = [0.1, 0.2, 0.3]
Q3 = [0.7, 0.8, 0.9]
Q4 = [1.0, 1.1, 1.2]

flows = [Q1, Q2, Q3, Q4]

shift_amount = 2
attenuation_factor = 0.5
lag_amount = 2

transform_params = [(0, None), (1, shift_amount), (3, attenuation_factor), (4, lag_amount)]


print(combine_streamflows(flows, transform_params))

#print(basin_network_dictionary['buckets'][0][1])
print(basin_network_dictionary['buckets'][0][0]['q_overflow'])
print(basin_network_dictionary['buckets'][0][1]['q_overflow'])

#print(basin_network_dictionary['buckets'][0][1]['q_spigot'])
print(combine_streamflows([basin_network_dictionary['buckets'][0][0]['q_overflow'], basin_network_dictionary['buckets'][0][1]['q_overflow']], [(0, None), (0, None)]))
'''

### 1.8 Visualizing a sample of the bucket fluxes
We plot the model simulations to ensure the existence of fluxes and validate that the generated values are realistics for both the spigot (channel flow) and over the top (flooding). 

In [ ]:
def viz_simulation(ibuc):
    fig = plt.figure(figsize=(12, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    print('Bucket:', ibuc)
    print("Overflow mean:", np.round(bucket_dictionary[ibuc].q_overflow.mean(),2))
    print("Overflow max:", np.round(bucket_dictionary[ibuc].q_overflow.max(),2))
    bucket_dictionary[ibuc].loc[:100,input_vars].plot(ax=ax1)
    bucket_dictionary[ibuc].loc[:100,output_vars].plot(ax=ax2)
    bucket_dictionary[ibuc].loc[:100,"h_bucket"].plot(ax=ax2)
    ax1.set_title('Model inputs')
    ax2.set_title('Model outputs')
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
for ibuc in buckets_for_val:
    viz_simulation(ibuc)

# 2. Deep learning model
This section sets up our deep learning model and training procedure.

### 2.1 Defining the neural network model
This is the part of the notebook where we will be using the previous simulations to learn from the generated data and subsequently make predictions. Here, we leverage the simulations to extract valuable insights and apply them in our learning and prediction processes.


Here we define a class called LSTM1, which is a PyTorch module for a single-layer Long Short-Term Memory (LSTM) network. 

**Brief explanation**
- The input to the module is a tensor x of shape (batch_size, seq_length, input_size), which represents a sequence of batch_size samples, each of length seq_length, with input_size features at each time step. 
- The LSTM layer is defined using the nn.LSTM class, with input_size as the size of the input layer, hidden_size as the size of the hidden state, and batch_first=True indicating that the first dimension of the input tensor is the batch size. 
- The output of the LSTM layer is passed through a ReLU activation function, and then to a fully connected layer (nn.Linear) with num_classes output units. 
- The forward method takes the input tensor x as an argument, along with an optional tuple init_states representing the initial hidden and internal states of the LSTM layer, and returns the output tensor prediction. 
- If init_states is not provided, it is initialized as a tensor of zeros with shape (batch_size, hidden_size).

In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, batch_size, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length 
        self.batch_size = batch_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.fc_1 =  nn.Linear(hidden_size, num_classes) #fully connected 1
   
    def forward(self, x, init_states=None):

        if init_states is None:
            h_t = Variable(torch.zeros(batch_size, self.hidden_size)) # hidden state
            c_t = Variable(torch.zeros(batch_size, self.hidden_size)) # internal state
        else:
            h_t, c_t = init_states
           
        out, _ = self.lstm(x)
        out = self.relu(out)
        prediction = self.fc_1(out) # Dense, fully connected layer
        
        return prediction

### 2.2 Defining a procedure for model validation
Here we verify that our model is working the way we expect. We would especially want to check model validation when changing hyperparameters.

We define a function that validates and tests the LSTM model, as well as checks the water balance of the system. 

**Brief explanation**
- We use the pre-defined LSTM model to make predictions on the validation data. 
- The output of this model is then used to compute two different metrics, the Nash-Sutcliffe Efficiency (NSE) for the spigot_out and mass_overflow columns of the dataframe.
- We plot the actual spigot_out and mass_overflow values against their corresponding LSTM predictions. 
- We check the water balance of the system by summing up the input, evapotranspiration, mass_overflow, spigot_out, and the last recorded water level in the dataframe, and compare this to the total mass out of or left in the system. 
- We print out the percent mass residual as a measure of how well the system is balanced.

In [ ]:
def check_validation_period(lstm, np_val_seq_X, ibuc, n_plot=100):
    
    def __make_prediction():
        lstm_output_val = lstm(torch.Tensor(np_val_seq_X[ibuc]).to(device=device))
        val_spigot_prediction = []
        val_overflow_prediction = []
        for i in range(lstm_output_val.shape[0]):
            val_spigot_prediction.append((lstm_output_val[i,-1,1].cpu().detach().numpy() * \
                                    np.std(df.loc[train_start:train_end,'q_spigot'])) + \
                                   np.mean(df.loc[train_start:train_end,'q_spigot']))

            val_overflow_prediction.append((lstm_output_val[i,-1,0].cpu().detach().numpy() * \
                                    np.std(df.loc[train_start:train_end,'q_overflow'])) + \
                                   np.mean(df.loc[train_start:train_end,'q_overflow']))
        return val_spigot_prediction, val_overflow_prediction
    
    def __compute_nse():
        spigot_out = df.loc[val_start:val_end, 'q_spigot']
        spigot_mean = np.mean(spigot_out)
        spigot_pred_variance = 0
        spigot_obs_variance = 0

        overflow_out = df.loc[val_start:val_end, 'q_overflow']
        overflow_mean = np.mean(overflow_out)
        overflow_pred_variance = 0
        overflow_obs_variance = 0

        for i, pred_spigot in enumerate(val_spigot_prediction):
            t = i + seq_length - 1
            spigot_pred_variance += np.power(( pred_spigot          - spigot_out.values[t]), 2)
            spigot_obs_variance  += np.power(( spigot_mean          - spigot_out.values[t]), 2)

        for i, pred_overflow in enumerate(val_overflow_prediction):
            t = i + seq_length - 1
            overflow_pred_variance += np.power((pred_overflow          - overflow_out.values[t]), 2)
            overflow_obs_variance  += np.power((overflow_mean          - overflow_out.values[t]), 2)
        spigot_nse = np.round( 1 - ( spigot_pred_variance / spigot_obs_variance   ), 4)
        overland_flow_nse = np.round( 1 - ( overflow_pred_variance / overflow_obs_variance ), 4)
        return spigot_nse, overland_flow_nse

    def __compute_mass_balance():
        mass_in = df.sum()['precip']
        mass_out = df.sum()['et'] + \
                   df.sum()['q_overflow'] + \
                   df.sum()['q_spigot'] + \
                   df.loc[num_records - 1, 'h_bucket']
        return mass_in, mass_out

        
    df = bucket_dictionary[ibuc]
    val_spigot_prediction, val_overflow_prediction = __make_prediction()
    spigot_nse, overland_flow_nse = __compute_nse()
    mass_in, mass_out = __compute_mass_balance()
        
    print("Spigot NSE", spigot_nse)
    print("Overflow NSE", overland_flow_nse)
    print("Mass into the system: ", mass_in)
    print("Mass out or left over:", mass_out)
    print("percent mass resudual: {:.0%}".format((mass_in - mass_out) /mass_in))
    
    fig = plt.figure(figsize=(12, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    ax1.plot(df.loc[val_start + seq_length-1:val_start+n_plot+seq_length - 1,'q_spigot'].values, label="Spigot out")
    ax1.plot(val_spigot_prediction[:n_plot], label="LSTM spigot out")
    ax1.legend()
    ax2.plot(df.loc[val_start+seq_length-1:val_start+n_plot+seq_length-1,'q_overflow'].values, label="Overflow")
    ax2.plot(val_overflow_prediction[:n_plot], label="LSTM Overflow")
    ax2.legend()
    plt.show()
    plt.close()

### 2.3 Instantiating the neural network model (LSTM)

Using the hyperparameters from Section 1.2, we define a specific instance of the LSTM model and set up the LSTM

In [ ]:
torch.manual_seed(1)
lstm = LSTM1(num_classes=n_output,  
             input_size=n_input,    
             hidden_size=hidden_state_size, 
             num_layers=num_layers, 
             batch_size=batch_size, 
             seq_length=seq_length).to(device=device)

### 2.4 Setting up the data to feed into the model

We will set up data for:
- training, to calculate the loss which is backpropogated through the model
- validation, where we get predictions from the trained model and see if the performance is up to our standards
- testing, the data we will utimately use to report the LSTM performance. 

Note: testing is the last thing we would do, if we go back to validation after this step, we would be P-hacking.

**Fitting a scaler to the training set to transform all the data**

Here we fit a scaler to the training set, allowing for the transformation of input and output variables to a normalized and standardized scale, which helps in training the model and maintaining consistency across different datasets. This normalization step ensures that the data is suitable for training our LSTM model. 

In [ ]:
def fit_scaler():
    frames = [bucket_dictionary[ibuc].loc[train_start:train_end, input_vars] for ibuc in buckets_for_training]
    df_in = pd.concat(frames)    
    scaler_in = StandardScaler()
    scaler_train_in = scaler_in.fit_transform(df_in)

    frames = [bucket_dictionary[ibuc].loc[train_start:train_end, output_vars] for ibuc in buckets_for_training]
    df_out = pd.concat(frames)    
    scaler_out = StandardScaler()
    scaler_train_out = scaler_out.fit_transform(df_out)
    return scaler_in, scaler_out

In [ ]:
scaler_in, scaler_out = fit_scaler()

**Function to create data loader for each data split**

We prepare and organize the data for training. We create data loaders that handle batch processing and shuffling of the data. We also add some preprocessing steps such as scaling the input and output variables. The data loaders and numpy arrays are used for feeding the data into the neural network during the training process.

In [ ]:
def make_data_loader(start, end, bucket_list):
    loader = {}
    np_seq_X = {}
    np_seq_y = {}
    for ibuc in bucket_list:
        df = bucket_dictionary[ibuc]
        scaler_in_i = scaler_in.transform(df.loc[start:end, input_vars])
        scaler_out_i = scaler_out.transform(df.loc[start:end, output_vars])
        np_seq_X[ibuc] = np.zeros((scaler_in_i.shape[0] - seq_length, seq_length, n_input))
        np_seq_y[ibuc] = np.zeros((scaler_out_i.shape[0] - seq_length, seq_length, n_output))
        for i in range(0, scaler_in_i.shape[0] - seq_length):
            t = i+seq_length
            np_seq_X[ibuc][i, :, :] = scaler_in_i[i:t,:]
            np_seq_y[ibuc][i, :, :] = scaler_out_i[i:t,:]

        ds = torch.utils.data.TensorDataset(torch.Tensor(np_seq_X[ibuc]), 
                                                  torch.Tensor(np_seq_y[ibuc]))
        loader[ibuc] = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True)
    return loader, np_seq_X, np_seq_y

We use the function above and parameters defined in the notebook environment to generate the training, validation, test data loaders

In [ ]:
train_loader, np_train_seq_X, np_train_seq_y = make_data_loader(train_start, train_end, buckets_for_training)
val_loader, np_val_seq_X, np_val_seq_y = make_data_loader(val_start, val_end, buckets_for_val)
test_loader, np_test_seq_X, np_test_seq_y = make_data_loader(test_start, test_end, buckets_for_test)

### 2.5 Training the model: Learning the general response of the example dynamic ''hydrologic" system
Now is the time to train the model! Everything above was done in preparation for this step.

Here we define a function to train the LSTM neural network model with the nn.MSELoss() loss function and using the Adam optimizer and hyperparameters defined above.

**Brief explanation**:
- The training is done for a specified number of epochs.
- For each epoch, the training data is divided into buckets. 
- For each bucket, the data is loaded using a PyTorch DataLoader and passed through the LSTM model. 
- The output is then compared with the target values using the custom loss function. 
- The gradients are calculated and the optimizer is used to update the weights of the model. 
- We use the tqdm library to show the progress of the training. 
- Finally, we estimate the average RMSE for each epoch. 

In [ ]:
def train_model(lstm, train_loader, buckets_for_training):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(lstm.parameters(), lr=learning_rate[0])
    epoch_bar = tqdm(range(num_epochs),desc="Training", position=0, total=num_epochs)
    
    # Create a dictionary to store the results
    results = {}
    for epoch in epoch_bar:

        for ibuc in buckets_for_training:

            batch_bar = tqdm(enumerate(train_loader[ibuc]),
                             desc="Bucket: {}, Epoch: {}".format(str(ibuc),str(epoch)),
                             position=1,
                             total=len(train_loader[ibuc]), leave=False, disable=True)

            for i, (data, targets) in batch_bar:

                optimizer.zero_grad()

                optimizer = optim.Adam(lstm.parameters(), lr=learning_rate[epoch])

                data = data.to(device=device)
                targets = targets.to(device=device)

                # Forward
                lstm_output = lstm(data) 
                loss = criterion(lstm_output, targets)

                #backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()

                batch_bar.set_postfix(loss=loss.to(device).item(),
                                      RMSE="{:.2f}".format(loss**(1/2)),
                                      epoch=epoch)
                batch_bar.update()

            with torch.no_grad():
                rmse_list = []
                for i, (data_, targets_) in enumerate(train_loader[ibuc]):
                    data_ = data_.to(device=device)
                    targets_ = targets_.to(device=device)
                    lstm_output_ = lstm(data_)
                    MSE_ = criterion(lstm_output_, targets_)
                    rmse_list.append(MSE_**(1/2))

            meanrmse = np.mean(np.array(torch.Tensor(rmse_list)))
            epoch_bar.set_postfix(loss=loss.cpu().item(),
                                  RMSE="{:.2f}".format(meanrmse),
                                  epoch=epoch)
            
            if ibuc not in results:
                results[ibuc] = {"loss": [], "RMSE": []}
            results[ibuc]["loss"].append(loss.cpu().item())
            results[ibuc]["RMSE"].append(meanrmse)
            #....todo..? also add IT metrics in results in this function ...
            batch_bar.update()

        𝗰𝗹𝗲𝗮𝗿_𝗼𝘂𝘁𝗽𝘂𝘁
        
    return lstm, results

Run the train model functon by prescribing the buckets and data to use for training and the intantiated LSTM

In [ ]:
lstm, results = train_model(lstm, train_loader, buckets_for_training)

### 2.5 Visualizing the learning curves

 We plot the loss and root mean square error (RMSE) metrics for each epoch to check if the model fitting has converged

In [ ]:
def viz_learning_curve(results):
    fig = plt.figure(figsize=(12, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    for ibuc in buckets_for_training:
        ax1.plot(range(num_epochs), results[ibuc]['loss'])
        ax2.plot(range(num_epochs), results[ibuc]['RMSE'])
    ax1.set_ylabel('Loss')
    ax2.set_ylabel('RMSE')
    ax1.set_xlabel('Epoch')
    ax2.set_xlabel('Epoch')
    plt.suptitle("Learning curves for each bucket") 
    plt.show()

In [ ]:
viz_learning_curve(results)

### 2.6 Checking that the model works on the validation data
Now that we have a trained model, we can see how it works on our validation split.

In [ ]:
for ibuc in buckets_for_val:
    check_validation_period(lstm, np_val_seq_X, ibuc)

# 3. Experimentation

Now that we have a general leaky bucket model setup and training/validation framework, we can use it to experiment with variations on the bucket characteristics and modelling setup.


Try changing some bucket configuration attributes here to modify the "ground truth" data, for example:
- to simulate a more "flashy" system, reduce the probability of heavy precipitation and increase the magnitude
- to simulate smaller buckets, reduce the size of the bucket attributes
- or to add more noise, increase noise from 0.1 to different values 

Try changing some aspects of the modelling setup, for example:
- increase or decrease the number of training buckets
- increase or decrease the timeseries length

In [ ]:
rain_probability_range = {"None": [0.6, 0.7], "Light": [0.5, 0.8], "Heavy": [0.05, 0.1]}
rain_depth_range = {"Light": [0, 2], "Heavy": [6, 14]}

bucket_attributes_range = {"A_bucket": [1.0, 2.0],
                           "A_spigot": [0.1, 0.2],
                           "H_bucket": [5.0, 6.0],
                           "H_spigot": [1.0, 3.0],
                           "K_infiltration": [1e-7, 1e-9],
                           "ET_parameter": [7, 9]
                          }

noise = {"pet": 0.1, "et": 0.1, "q": 0.1, "head": 0.1} 

In [ ]:
n_buckets_split = {"train": 25, "val": 5,"test": 1}
time_splits = {"train": 2000, "val": 1000,"test": 500}
num_records = time_splits["train"] + time_splits["val"] + time_splits["test"] + seq_length * 3
n_buckets = n_buckets_split["train"] + n_buckets_split["val"] + n_buckets_split["test"]

[[buckets_for_training, train_start, train_end],
[buckets_for_val, val_start, val_end],
[buckets_for_test, test_start, test_end]]= split_parameters()

Re-generate forcings and 'ground truth' data for buckets

In [ ]:
buckets, h_water_level, mass_overflow = setup_buckets(n_buckets)

in_list = {}
for ibuc in range(n_buckets):
    bucket_rain_params = pick_rain_params()
    in_list[ibuc] = [0]
    for i in range(1, num_records):
        in_list[ibuc].append(random_rain(in_list[ibuc][i-1], bucket_rain_params))

bucket_dictionary = {}
for ibuc in range(n_buckets):
    bucket_dictionary[ibuc] = run_bucket_simulation(ibuc)
    
for ibuc in buckets_for_val:
    viz_simulation(ibuc)

Re-create new data loaders

In [ ]:
scaler_in, scaler_out = fit_scaler()
train_loader, np_train_seq_X, np_train_seq_y = make_data_loader(train_start, train_end, buckets_for_training)
val_loader, np_val_seq_X, np_val_seq_y = make_data_loader(val_start, val_end, buckets_for_val)
test_loader, np_test_seq_X, np_test_seq_y = make_data_loader(test_start, test_end, buckets_for_test)

Reinitializing the LSTM model, the training process is performed with the updated dataset.

In [ ]:
lstm = LSTM1(num_classes=n_output,  
             input_size=n_input,    
             hidden_size=hidden_state_size, 
             num_layers=num_layers, 
             batch_size=batch_size, 
             seq_length=seq_length).to(device=device)

lstm, results = train_model(lstm, train_loader, buckets_for_training)

Visualize the learning curve based on the training results.

In [ ]:
viz_learning_curve(results)

for ibuc in buckets_for_val:
    check_validation_period(lstm, np_val_seq_X, ibuc)